In [17]:
# ============================================================
# CONFIGURACIÓN E IMPORTS
# ============================================================

import re
import pandas as pd
from datetime import datetime, timedelta
from pathlib import Path
import sys
import os

# Agregar la raíz del proyecto al path
ROOT = Path().resolve().parent
sys.path.insert(0, str(ROOT))

# Importar configuración centralizada
from config import PATHS, CONFIG


In [ ]:
# ============= CONFIGURACIÓN USANDO CONFIG.PY =============

# Usar rutas centralizadas
CARPETA_DATOS_CRUDOS = PATHS.RAW_WA
CARPETA_PROCESADOS = PATHS.UNCLEANED

# Configuración del participante a extraer
PARTICIPANTE_SELECCIONADO = "Nombre Participante"        # Nombre del participante en los chats de WhatsApp
PSEUDONIMO = "EST001"                           # Pseudónimo para el participante

# Usar configuración centralizada para fechas
USAR_FILTRO_FECHA = CONFIG.USAR_FILTRO_FECHA
MESES_ATRAS = CONFIG.MESES_ATRAS
FECHA_DESDE = CONFIG.FECHA_DESDE

print("="*60)
print("CONFIGURACIÓN ACTIVA:")
print("="*60)
print(f"Datos crudos: {CARPETA_DATOS_CRUDOS}")
print(f"Procesados: {CARPETA_PROCESADOS}")
print(f"Participante: {PARTICIPANTE_SELECCIONADO} → {PSEUDONIMO}")
print(f"Filtro de fecha: {'Activo' if USAR_FILTRO_FECHA else 'Inactivo'}")
if USAR_FILTRO_FECHA:
    if FECHA_DESDE:
        print(f"  Desde: {FECHA_DESDE}")
    else:
        print(f"  Últimos {MESES_ATRAS} meses")
print("="*60)

In [20]:
def normalizar_fecha(fecha_str):
    """
    Normaliza diferentes formatos de fecha a YYYY-MM-DD HH:MM:SS
    """
    if pd.isna(fecha_str) or fecha_str == "":
        return ""
    
    # Si ya es un objeto datetime
    if isinstance(fecha_str, datetime):
        return fecha_str.strftime('%Y-%m-%d %H:%M:%S')
    
    fecha_str = str(fecha_str).strip()

    # Normalizar espacios invisibles y múltiple espacio (ej. NBSP U+202F, \xa0)
    fecha_str = fecha_str.replace('\u202f', ' ').replace('\xa0', ' ')
    fecha_str = re.sub(r'\s+', ' ', fecha_str)

    # Normalizar variantes de a. m./p. m. a formas AM/PM más fáciles de detectar
    fecha_str = re.sub(r'(?i)\ba\.?\s*m\.?\b', 'AM', fecha_str)
    fecha_str = re.sub(r'(?i)\bp\.?\s*m\.?\b', 'PM', fecha_str)

    try:
        # Formato ISO con timezone (2023-01-16T11:34:28Z)
        if 'T' in fecha_str:
            fecha_str = fecha_str.replace('Z', '')
            dt = datetime.fromisoformat(fecha_str.split('+')[0].split('-0')[0])
            return dt.strftime('%Y-%m-%d %H:%M:%S')
        
        # Formato WhatsApp español con COMA: "8/9/2025, 10:48 AM" (ya normalizado a AM/PM)
        if 'PM' in fecha_str or 'AM' in fecha_str:
            # Remover coma si existe
            fecha_str_limpia = fecha_str.replace(',', '')
            partes = fecha_str_limpia.split()
            
            if len(partes) >= 2:
                fecha_parte = partes[0]  # "8/9/2025"
                hora_parte = partes[1]   # "10:48"
                am_pm = partes[2] if len(partes) > 2 else "AM"  # "PM" o "AM"
                
                # Parsear fecha (puede ser D/M/YY o DD/MM/YY o M/D/YYYY)
                fecha_nums = fecha_parte.split('/')
                try:
                    # Determinar formato: si el tercer número es >31, es año completo
                    if len(fecha_nums) == 3:
                        if int(fecha_nums[2]) > 31:
                            # Formato M/D/YYYY
                            mes = int(fecha_nums[0])
                            dia = int(fecha_nums[1])
                            anio = int(fecha_nums[2])
                        else:
                            # Formato D/M/YY
                            dia = int(fecha_nums[0])
                            mes = int(fecha_nums[1])
                            anio = int(fecha_nums[2])
                            # Ajustar año (25 -> 2025, 24 -> 2024)
                            if anio < 100:
                                anio = 2000 + anio if anio < 50 else 1900 + anio
                    else:
                        dia = mes = anio = None
                except Exception:
                    dia = mes = anio = None
                
                if dia is not None and mes is not None and anio is not None:
                    # Parsear hora
                    hora_nums = hora_parte.split(':')
                    hora = int(hora_nums[0])
                    minuto = int(hora_nums[1]) if len(hora_nums) > 1 else 0
                    
                    # Convertir a formato 24 horas
                    if am_pm.upper() == 'PM' and hora != 12:
                        hora += 12
                    elif am_pm.upper() == 'AM' and hora == 12:
                        hora = 0
                    
                    dt = datetime(anio, mes, dia, hora, minuto, 0)
                    return dt.strftime('%Y-%m-%d %H:%M:%S')
        
        # Formato M/D/YYYY (7/17/2025)
        if '/' in fecha_str and len(fecha_str.split('/')) == 3:
            parts = fecha_str.split('/')
            if len(parts[2]) == 4:  # Año completo
                try:
                    dt = datetime.strptime(fecha_str, '%m/%d/%Y')
                    return dt.strftime('%Y-%m-%d %H:%M:%S')
                except Exception:
                    pass
        
        # Formato español: "ene 16, 2023 11:34:28 am"
        meses_es = {
            'ene': 'Jan', 'feb': 'Feb', 'mar': 'Mar', 'abr': 'Apr',
            'may': 'May', 'jun': 'Jun', 'jul': 'Jul', 'ago': 'Aug',
            'sep': 'Sep', 'oct': 'Oct', 'nov': 'Nov', 'dic': 'Dec'
        }
        fecha_lower = fecha_str.lower()
        for mes_es, mes_en in meses_es.items():
            if mes_es in fecha_lower:
                fecha_str = fecha_str.lower().replace(mes_es, mes_en)
                break
        
        # Intentar parsear con varios formatos comunes
        formatos = [
            '%b %d, %Y %I:%M:%S %p',  # Jan 16, 2023 11:34:28 AM
            '%B %d, %Y %I:%M:%S %p',  # January 16, 2023 11:34:28 AM
            '%Y-%m-%d %H:%M:%S',      # 2023-01-16 11:34:28
            '%d/%m/%Y %H:%M:%S',      # 16/01/2023 11:34:28
            '%m/%d/%Y %H:%M:%S',      # 01/16/2023 11:34:28
            '%Y-%m-%d',               # 2023-01-16
            '%d/%m/%Y',               # 16/01/2023
            '%m/%d/%Y',               # 01/16/2023
        ]
        
        for formato in formatos:
            try:
                dt = datetime.strptime(fecha_str, formato)
                return dt.strftime('%Y-%m-%d %H:%M:%S')
            except:
                continue
        
        # Si nada funciona, retornar la fecha original
        return fecha_str
        
    except Exception as e:
        # En caso de error, retornar la fecha original
        return fecha_str


In [21]:
def extraer_mensajes_participante(archivo_txt, participante, pseudonimo, fecha_limite=None, inicio_idx=0, mensajes_existentes=None):
    """
    Extrae todos los mensajes de un participante específico de una conversación de WhatsApp
    
    Args:
        archivo_txt: Ruta del archivo .txt con la conversación
        participante: Nombre del participante a filtrar
        pseudonimo: Pseudónimo para anonimizar al participante
        fecha_limite: Fecha mínima para filtrar mensajes (datetime o None)
        inicio_idx: Índice inicial para la numeración (para continuar numeración)
        mensajes_existentes: DataFrame con mensajes ya existentes (para evitar duplicados)
    
    Returns:
        DataFrame con los mensajes extraídos
    """
    
    # Patrones para mensajes de WhatsApp en diferentes formatos
    # Formato 1: CON COMA: "DD/M/YY, H:MM p. m. - Nombre: Mensaje"
    patron_con_coma = r'(\d{1,2}/\d{1,2}/\d{2,4}),\s+(\d{1,2}:\d{2}\s+[ap]\.\s+m\.)\s+-\s+([^:]+):\s+(.*)'
    
    # Formato 2: SIN COMA: "DD/M/YY H:MM p. m. - Nombre: Mensaje"
    patron_sin_coma = r'(\d{1,2}/\d{1,2}/\d{2,4})\s+(\d{1,2}:\d{2}\s+[ap]\.\s+m\.)\s+-\s+([^:]+):\s+(.*)'
    
    mensajes = []
    
    # Crear conjunto de mensajes existentes para búsqueda rápida (fecha + primeros 100 chars del texto)
    mensajes_existentes_set = set()
    if mensajes_existentes is not None and not mensajes_existentes.empty:
        for _, row in mensajes_existentes.iterrows():
            # Validar que los valores no sean NaN o nulos
            fecha_pub = row.get('fecha_publicacion', '')
            texto_pub = row.get('texto_publicacion', '')
            
            # Convertir a string y manejar valores nulos
            if pd.notna(fecha_pub) and pd.notna(texto_pub):
                fecha_str = str(fecha_pub)
                texto_str = str(texto_pub)
                clave = f"{fecha_str}_{texto_str[:100]}"
                mensajes_existentes_set.add(clave)
    
    try:
        with open(archivo_txt, 'r', encoding='utf-8') as archivo:
            lineas = archivo.readlines()
            
            mensaje_actual = None
            
            for linea in lineas:
                # Intentar ambos patrones
                match = re.match(patron_con_coma, linea)
                tiene_coma = True
                
                if not match:
                    match = re.match(patron_sin_coma, linea)
                    tiene_coma = False
                
                if match:
                    # Si hay un mensaje anterior y es del participante, guardarlo
                    if mensaje_actual and mensaje_actual['nombre'] == participante:
                        # Aplicar filtro de fecha si está activo. Si no pudimos parsear la fecha
                        # (fecha_dt es None) y hay un filtro, NO incluir el mensaje.
                        if fecha_limite is None or (mensaje_actual['fecha_dt'] is not None and mensaje_actual['fecha_dt'] >= fecha_limite):
                            # Verificar si el mensaje ya existe
                            texto_limpio = mensaje_actual['texto'].strip()
                            clave = f"{mensaje_actual['fecha']}_{texto_limpio[:100]}"
                            
                            if clave not in mensajes_existentes_set:
                                idx = inicio_idx + len(mensajes)
                                mensajes.append({
                                    'id_participante': pseudonimo,
                                    'id_publicacion': f"{pseudonimo}_{idx+1:03d}",
                                    'fuente': 'WhatsApp',
                                    'fecha_publicacion': mensaje_actual['fecha'],
                                    'texto_publicacion': texto_limpio
                                })
                    
                    # Procesar nuevo mensaje
                    fecha_str = match.group(1)
                    hora_str = match.group(2)
                    nombre = match.group(3).strip()
                    texto = match.group(4).strip()
                    
                    # Combinar fecha y hora (mantener formato original con o sin coma)
                    if tiene_coma:
                        fecha_hora = f"{fecha_str}, {hora_str}"
                    else:
                        fecha_hora = f"{fecha_str} {hora_str}"
                    
                    # Normalizar fecha
                    fecha_normalizada = normalizar_fecha(fecha_hora)
                    
                    # Convertir a datetime para comparación (más robusto)
                    fecha_dt = None
                    try:
                        # Intentar con fecha y hora
                        fecha_dt = datetime.strptime(fecha_normalizada, '%Y-%m-%d %H:%M:%S')
                    except:
                        try:
                            # Intentar sólo fecha (sin hora)
                            fecha_dt = datetime.strptime(fecha_normalizada, '%Y-%m-%d')
                        except Exception:
                            # No se pudo parsear la fecha; dejamos fecha_dt en None
                            fecha_dt = None
                    
                    mensaje_actual = {
                        'nombre': nombre,
                        'fecha': fecha_normalizada,
                        'fecha_dt': fecha_dt,
                        'texto': texto,
                    }
                else:
                    # Línea de continuación del mensaje anterior
                    if mensaje_actual:
                        mensaje_actual['texto'] += ' ' + linea.strip()
            
            # No olvidar el último mensaje si es del participante
            if mensaje_actual and mensaje_actual['nombre'] == participante:
                # Aplicar filtro de fecha si está activo. Si no pudimos parsear la fecha
                # (fecha_dt es None) y hay un filtro, NO incluir el mensaje.
                if fecha_limite is None or (mensaje_actual['fecha_dt'] is not None and mensaje_actual['fecha_dt'] >= fecha_limite):
                    # Verificar si el mensaje ya existe
                    texto_limpio = mensaje_actual['texto'].strip()
                    clave = f"{mensaje_actual['fecha']}_{texto_limpio[:100]}"
                    
                    if clave not in mensajes_existentes_set:
                        idx = inicio_idx + len(mensajes)
                        mensajes.append({
                            'id_participante': pseudonimo,
                            'id_publicacion': f"{pseudonimo}_{idx+1:03d}",
                            'fuente': 'WhatsApp',
                            'fecha_publicacion': mensaje_actual['fecha'],
                            'texto_publicacion': texto_limpio
                        })
    
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo '{archivo_txt}'")
        return pd.DataFrame()
    except Exception as e:
        print(f"Error al procesar el archivo: {e}")
        return pd.DataFrame()
    
    # Crear DataFrame
    df = pd.DataFrame(mensajes)
    
    return df


In [ ]:
def main():
    print("=" * 80)
    print("EXTRACTOR DE MENSAJES DE WHATSAPP - MÚLTIPLES ARCHIVOS")
    print("=" * 80)
    
    # Crear carpeta de procesados si no existe
    if not os.path.exists(CARPETA_PROCESADOS):
        os.makedirs(CARPETA_PROCESADOS)
        print(f"✓ Carpeta '{CARPETA_PROCESADOS}/' creada")
    
    # Calcular fecha límite si el filtro está activo
    fecha_limite = None
    if USAR_FILTRO_FECHA:
        if FECHA_DESDE:
            fecha_limite = datetime.strptime(FECHA_DESDE, '%Y-%m-%d')
            print(f"\nFiltro: Mensajes desde {FECHA_DESDE}")
        else:
            fecha_limite = datetime.now() - timedelta(days=MESES_ATRAS * 30)
            print(f"\nFiltro: Mensajes de los últimos {MESES_ATRAS} meses")
            print(f"   (Desde: {fecha_limite.strftime('%Y-%m-%d')})")
    else:
        print("\nSin filtro de fecha - Extrayendo todos los mensajes")
    
    # Verificar carpeta
    if not os.path.exists(CARPETA_DATOS_CRUDOS):
        print(f"\nError: La carpeta '{CARPETA_DATOS_CRUDOS}/' no existe")
        return None
    
    # Buscar todos los archivos .txt en la carpeta
    archivos_txt = sorted([
        f for f in os.listdir(CARPETA_DATOS_CRUDOS)
        if f.endswith('.txt') and os.path.isfile(os.path.join(CARPETA_DATOS_CRUDOS, f))
        and not f.startswith('.')  # Ignorar archivos ocultos
    ])
    
    if not archivos_txt:
        print(f"\nNo se encontraron archivos .txt en '{CARPETA_DATOS_CRUDOS}/'")
        return None
    
    print(f"\nArchivos .txt encontrados: {len(archivos_txt)}")
    print(f"Buscando mensajes de: {PARTICIPANTE_SELECCIONADO}")
    print("-" * 80)
    
    # Verificar si ya existe el archivo consolidado
    RUTA_SALIDA = CARPETA_PROCESADOS / f"{PSEUDONIMO}.csv"
    df_existente = pd.DataFrame()
    inicio_idx = 0
    
    if os.path.exists(RUTA_SALIDA):
        try:
            df_existente = pd.read_csv(RUTA_SALIDA, encoding='utf-8-sig')
            inicio_idx = len(df_existente)
            print(f"\nArchivo existente encontrado: {len(df_existente)} mensajes")
            print(f"   Continuando desde índice: {inicio_idx + 1}")
            # Si el archivo existente no tiene columna 'fuente', marcar como 'Unknown'
            if not df_existente.empty and 'fuente' not in df_existente.columns:
                df_existente['fuente'] = 'Unknown'
        except Exception as e:
            print(f"\nError al leer archivo existente: {e}")
            print("   Se creará un archivo nuevo.")
            df_existente = pd.DataFrame()
            inicio_idx = 0
    else:
        print(f"\nNo existe archivo previo. Creando nuevo archivo.")
    
    # Procesar cada archivo
    todos_mensajes_nuevos = []
    estadisticas = []
    
    for idx, nombre_archivo in enumerate(archivos_txt, 1):
        ruta_archivo = os.path.join(CARPETA_DATOS_CRUDOS, nombre_archivo)
        
        print(f"\n[{idx}/{len(archivos_txt)}] Procesando: {nombre_archivo}")
        
        try:
            # Extraer mensajes de este archivo
            df_mensajes = extraer_mensajes_participante(
                ruta_archivo,
                PARTICIPANTE_SELECCIONADO,
                PSEUDONIMO,
                fecha_limite,
                inicio_idx + len(todos_mensajes_nuevos),
                df_existente
            )
            
            if not df_mensajes.empty:
                # Filtrar duplicados comparando con mensajes ya procesados
                if todos_mensajes_nuevos:
                    df_temp = pd.DataFrame(todos_mensajes_nuevos)
                    # Crear clave única para comparación
                    df_mensajes['clave'] = df_mensajes['fecha_publicacion'] + '_' + df_mensajes['texto_publicacion'].str[:100]
                    df_temp['clave'] = df_temp['fecha_publicacion'] + '_' + df_temp['texto_publicacion'].str[:100]
                    
                    # Filtrar mensajes que no estén en la lista temporal
                    df_mensajes = df_mensajes[~df_mensajes['clave'].isin(df_temp['clave'])]
                    df_mensajes = df_mensajes.drop('clave', axis=1)
                
                if not df_mensajes.empty:
                    todos_mensajes_nuevos.extend(df_mensajes.to_dict('records'))
                    print(f"    ✓ {len(df_mensajes)} mensajes nuevos extraídos")
                    
                    estadisticas.append({
                        'archivo': nombre_archivo,
                        'mensajes_nuevos': len(df_mensajes)
                    })
                else:
                    print(f"    ℹ️  Mensajes duplicados, ya procesados anteriormente")
            else:
                print(f"    ℹ️  Sin mensajes del participante o fuera del rango de fechas")
        
        except Exception as e:
            print(f"    Error: {e}")
            continue
    
    # Consolidar todos los mensajes
    if not todos_mensajes_nuevos:
        if df_existente.empty:
            print("\nNo se encontraron mensajes del participante en ningún archivo.")
            if USAR_FILTRO_FECHA:
                print("    Intenta desactivar el filtro de fecha o ajustar la fecha límite.")
        else:
            print("\nNo se encontraron mensajes nuevos para agregar.")
            print(f"   Total de mensajes en el archivo: {len(df_existente)}")
        return
    
    df_mensajes_nuevos = pd.DataFrame(todos_mensajes_nuevos)
    # Marcar la fuente para los mensajes recién extraídos
    df_mensajes_nuevos['fuente'] = 'WhatsApp'
    
    # Renumerar IDs de forma consecutiva
    for idx, row_idx in enumerate(df_mensajes_nuevos.index):
        nueva_idx = inicio_idx + idx
        df_mensajes_nuevos.at[row_idx, 'id_publicacion'] = f"{PSEUDONIMO}_{nueva_idx+1:03d}"
    
    # Combinar con existentes
    if not df_existente.empty:
        df_final = pd.concat([df_existente, df_mensajes_nuevos], ignore_index=True)
        print(f"\n✓ Total de mensajes nuevos agregados: {len(df_mensajes_nuevos)}")
        print(f"✓ Total de mensajes en archivo: {len(df_final)}")
    else:
        df_final = df_mensajes_nuevos
        print(f"\n✓ Extracción completada exitosamente!")
        print(f"✓ Total de mensajes extraídos: {len(df_final)}")
    
    # Ordenar por id_participante (numérico si es posible) antes de guardar
    if 'id_participante' in df_final.columns:
        try:
            nums = df_final['id_participante'].astype(str).str.extract(r'(\d+)$')[0]
            if nums.notna().any():
                df_final['_orden'] = nums.astype(float).astype('Int64')
                df_final = df_final.sort_values(['_orden', 'id_participante']).drop(columns=['_orden'])
            else:
                df_final = df_final.sort_values('id_participante')
        except Exception:
            df_final = df_final.sort_values('id_participante')

    # Guardar archivo consolidado
    df_final.to_csv(RUTA_SALIDA, index=False, encoding='utf-8-sig')
    print(f"✓ Archivo guardado como: {RUTA_SALIDA}")
    
    # Mostrar resumen por archivo
    if estadisticas:
        print("\n" + "=" * 80)
        print("RESUMEN POR ARCHIVO:")
        print("=" * 80)
        df_stats = pd.DataFrame(estadisticas)
        print(df_stats.to_string(index=False))
    
    # Mostrar estadísticas generales
    print("\n" + "=" * 80)
    print("ESTADÍSTICAS GENERALES:")
    print("=" * 80)
    print(f"  - Archivos procesados: {len(archivos_txt)}")
    print(f"  - Archivos con mensajes: {len(estadisticas)}")
    print(f"  - Total mensajes extraídos: {len(df_final)}")
    if not df_final.empty:
        print(f"  - Primera publicación: {df_final['fecha_publicacion'].iloc[0]}")
        print(f"  - Última publicación: {df_final['fecha_publicacion'].iloc[-1]}")
        print(f"  - Promedio de caracteres: {df_final['texto_publicacion'].str.len().mean():.0f}")
    print("=" * 80)
    
    if not df_mensajes_nuevos.empty:
        print("\nÚltimos 5 mensajes agregados:")
        print("=" * 80)
        print(df_mensajes_nuevos.tail())

if __name__ == "__main__":
    main()
